In [ ]:
import pandas as pd

def hapus_kolom_tidak_digunakan(file_input, file_output, kolom_yang_digunakan):
    # Membaca file CSV dengan memisahkan kolom berdasarkan delimiter ';'
    df = pd.read_csv(file_input, delimiter=';')

    # Mengecek nama kolom yang ada dalam DataFrame
    print("Nama Kolom dalam DataFrame:", df.columns)

    try:
        # Menghapus kolom yang tidak digunakan
        df = df[kolom_yang_digunakan]

        # Menyimpan DataFrame yang sudah dimodifikasi ke file CSV baru
        df.to_csv(file_output, index=False)

        print(f"File {file_output} telah berhasil dibuat dengan kolom yang diinginkan.")
    except KeyError as e:
        print(f"Error: {e}. Pastikan nama kolom yang digunakan sesuai dengan nama kolom dalam file CSV.")

if __name__ == "__main__":
    # Ganti nama file_input.csv dengan nama file CSV yang ingin Anda proses
    file_input = 'dataset.csv'

    # Ganti nama file_output.csv dengan nama file CSV yang ingin Anda hasilkan
    file_output = 'datasetfix.csv'

    # Ganti kolom_yang_digunakan dengan daftar kolom yang ingin Anda pertahankan
    kolom_yang_digunakan = ['Sentiment', 'text_tweet']

    # Memanggil fungsi untuk menghapus kolom yang tidak digunakan
    hapus_kolom_tidak_digunakan(file_input, file_output, kolom_yang_digunakan)


In [23]:
import numpy as np

TWEET_DATA = pd.read_csv("datasetfix.csv", encoding = "ISO-8859-1")
TWEET_DATA['tweet'].str.encode('ascii', 'ignore')
TWEET_DATA.head()

,Sentiment,tweet
0,positive,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,positive,Selamat berbuka puasa Semoga amal ibadah hari ...
2,positive,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,positive,selamat ya mas @adietaufan masuk hitamputih
4,positive,Asiknya nonton Hitam Putih Trans7


In [24]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].str.lower()


print('Case Folding Result : \n')
print(TWEET_DATA['tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    undang @n_shanijkt48 ke hitamputih, pemenang s...
1    selamat berbuka puasa semoga amal ibadah hari ...
2    ada nih di trans7 hitam putih, dia dpt penghar...
3          selamat ya mas @adietaufan masuk hitamputih
4                    asiknya nonton hitam putih trans7
Name: tweet, dtype: object






In [25]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(TWEET_DATA['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [undang, shanijkt, ke, hitamputih, pemenang, s...
1    [selamat, berbuka, puasa, semoga, amal, ibadah...
2    [ada, nih, di, trans, hitam, putih, dia, dpt, ...
3                [selamat, ya, mas, masuk, hitamputih]
4               [asiknya, nonton, hitam, putih, trans]
Name: tweet_tokens, dtype: object






In [26]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

TWEET_DATA['tweet_tokens_fdist'] = TWEET_DATA['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(TWEET_DATA['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(undang, 2), (shanijkt, 1), (ke, 1), (hitampu...
1    [(selamat, 1), (berbuka, 1), (puasa, 1), (semo...
2    [(di, 2), (ada, 1), (nih, 1), (trans, 1), (hit...
3    [(selamat, 1), (ya, 1), (mas, 1), (masuk, 1), ...
4    [(asiknya, 1), (nonton, 1), (hitam, 1), (putih...
Name: tweet_tokens_fdist, dtype: object


In [29]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
len(list_stopwords)
# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopword.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
len(list_stopwords)
# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

TWEET_DATA['tweet_tokens_WSW'] = TWEET_DATA['tweet_tokens'].apply(stopwords_removal) 


print(TWEET_DATA['tweet_tokens_WSW'].head())

758
0    [undang, shanijkt, hitamputih, pemenang, ssk, ...
1    [selamat, berbuka, puasa, semoga, amal, ibadah...
2         [trans, hitam, putih, penghargaan, norwegia]
3                                [selamat, hitamputih]
4               [asiknya, nonton, hitam, putih, trans]
Name: tweet_tokens_WSW, dtype: object


In [37]:
import spacy
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Load model bahasa Indonesia dari spaCy
nlp_id = spacy.load("xx_ent_wiki_sm")

# Membuat objek Stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Normalisasi kata dengan spaCy Bahasa Indonesia
def normalized_term_id(document):
    # Tokenisasi dokumen menggunakan spaCy
    doc = nlp_id(document)
    
    # Normalisasi kata dalam dokumen
    normalized_document = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]
    
    return normalized_document

# Normalisasi kata dengan Sastrawi
def stemmed_wrapper(term):
    return stemmer.stem(term)

# Membuat kamus untuk normalisasi kata
term_dict = {}

# Normalisasi kata dengan spaCy dan Sastrawi
for document in TWEET_DATA['tweet_tokens_WSW']:
    normalized_document_id = normalized_term_id(' '.join(document))
    normalized_document_stemmed = [stemmed_wrapper(term) for term in normalized_document_id]
    
    for term in normalized_document_stemmed:
        if term not in term_dict:
            term_dict[term] = ' '

# Menampilkan jumlah kata yang sudah dinormalisasi dan di-stem
print(len(term_dict))
print("------------------------")

# Melakukan stemming pada kamus term_dict
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term, ":", term_dict[term])

print(term_dict)
print("------------------------")

# Menerapkan term yang sudah di-stem pada dataframe
def get_stemmed_term(document):
    return [term_dict.get(term, term) for term in document]


TWEET_DATA['tweet_tokens_stemmed'] = TWEET_DATA['tweet_tokens_WSW'].apply(get_stemmed_term)
print(TWEET_DATA['tweet_tokens_stemmed'])


1
------------------------
 : 
{'': ''}
------------------------
0      [undang, shanijkt, hitamputih, pemenang, ssk, ...
1      [selamat, berbuka, puasa, semoga, amal, ibadah...
2           [trans, hitam, putih, penghargaan, norwegia]
3                                  [selamat, hitamputih]
4                 [asiknya, nonton, hitam, putih, trans]
                             ...                        
395            [banget, kesel, debat, pake, emosi, gini]
396            [miskin, miskin, sekolah, pungutan, liar]
397           [emosi, cepat, tua, nonton, emosi, bicara]
398    [penampilan, kyk, preman, taunya, bkin, kisruh...
399                  [berbelitbelit, muter, buang, mutu]
Name: tweet_tokens_stemmed, Length: 400, dtype: object


In [45]:
TWEET_DATA.to_csv("Text_Preprocessing.csv")


In [49]:


# Membaca file CSV dan mengabaikan kolom indeks 'Unnamed: 0'
text_preprocessing_data = pd.read_csv('text_preprocessing.csv', index_col=0)

# Memilih hanya kolom 'sentiment' dan 'tweet_tokens_stemmed'
selected_columns = text_preprocessing_data[['Sentiment', 'tweet_tokens_stemmed']]

# Menyimpan DataFrame yang telah dipilih ke dalam file CSV baru
selected_columns.to_csv('final_dataset.csv', index=False)

# Menampilkan DataFrame yang telah dipilih
print(selected_columns)


    Sentiment                               tweet_tokens_stemmed
0    positive  ['undang', 'shanijkt', 'hitamputih', 'pemenang...
1    positive  ['selamat', 'berbuka', 'puasa', 'semoga', 'ama...
2    positive  ['trans', 'hitam', 'putih', 'penghargaan', 'no...
3    positive                          ['selamat', 'hitamputih']
4    positive   ['asiknya', 'nonton', 'hitam', 'putih', 'trans']
..        ...                                                ...
395  negative  ['banget', 'kesel', 'debat', 'pake', 'emosi', ...
396  negative  ['miskin', 'miskin', 'sekolah', 'pungutan', 'l...
397  negative  ['emosi', 'cepat', 'tua', 'nonton', 'emosi', '...
398  negative  ['penampilan', 'kyk', 'preman', 'taunya', 'bki...
399  negative        ['berbelitbelit', 'muter', 'buang', 'mutu']

[400 rows x 2 columns]
